In [ ]:
# check for the GPU provided in the runtime
!nvidia-smi

Sun Dec 25 19:15:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    28W /  70W |   9040MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# using quiet method for controlling the log
# for suppressing the colored errors and warning in the terminal
!pip install --quiet transformers==4.1.1
# pytorch lightning for smoother model training and data loading
#!pip install --quiet https://github.com/PyTorchLightning/pytorch-lightning/releases/download/1.2.6/pytorch-lightning-1.2.6.tar.gz 
!pip install -q pytorch-lightning  
# using HuggingFace tokenizers
!pip install --quiet tokenizers==0.9.4
# Google's sentencepiece
!pip install --quiet sentencepiece==0.1.94

In [ ]:
# argparse makes it easier to write user friendly command line interfaces
import argparse
# package for faster file name matching
import glob
# makiing directories for data 
import os
# reading json files as the data is present in json files
import json
# time module for calculating the model runtime
import time
# Allows writing status messages to a file
import logging
# generate random float numbers uniformly
import random
# regex module for text 
import re
# module provides various functions which work on 
# iterators too produce complex iterators
from itertools import chain
from string import punctuation

# pandas for data manipulation
import pandas as pd
# numpy for array operations
import numpy as np
# PyTorch
import torch
# provides various classes representing file system paths
# with appropriate semantics
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

# splitting the data 
from sklearn.model_selection import train_test_split
# ANSII color formatting for ouput in terminal
from termcolor import colored
# wrapping paragraphs into string
import textwrap

# model checkpoints in pretrained model
from pytorch_lightning.callbacks import ModelCheckpoint

'''
optimizer - AdamW
T5 Conditional Generator in which we'll give conditions
T5 tokenizer because it is fast
training the model without a learning rate
'''
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [ ]:
# Seeds all the processes including numpy torch and other imported modules.
pl.seed_everything(0)

INFO:lightning_lite.utilities.seed:Global seed set to 0


0

In [ ]:
# check the version provided by Lightning
import pytorch_lightning as pl
print(pl.__version__)

1.8.6


In [ ]:
import json

with open('/content/pqa_train.json', 'r') as f:
  data = json.load(f)



In [ ]:
data['data'][1].keys()

dict_keys(['title', 'paragraphs'])

In [ ]:
# len 
len(data['data'])

901

In [ ]:
# We have a list of dictionaries in the "data". We can explore the 0th element
data['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [ ]:
data['data'][1]['title']

'علوم رایانه'

In [ ]:
len(data['data'][0]['paragraphs'])

1

In [ ]:
questions = data['data'][1]['paragraphs']

In [ ]:
# datapoint sample
questions[0]

{'qas': [{'answers': [{'answer_start': 29,
     'answer_end': 141,
     'text': 'به مجموعهٔ مطالعاتی گفته می\u200cشود که به زیربناهای نظری، روش\u200cهای طراحی و ساخت و چگونگی استفاده از رایانه می\u200cپردازند'}],
   'question': 'تعریف علوم کامپیوترچیست؟',
   'is_impossible': False,
   'id': 11},
  {'answers': [{'answer_start': 142,
     'answer_end': 215,
     'text': 'رشته علوم کامپیوتر را می\u200cتوان به زیررشته\u200cهای نظری و عملی بسیاری تقسیم کرد'}],
   'question': 'رشته علوم کامپوتر را چگونه می توان تقسیم کرد؟',
   'is_impossible': False,
   'id': 12},
  {'answers': [{'answer_start': 507,
     'answer_end': 543,
     'text': ' بر چالش\u200cهای موجود در اجرای محاسبات '}],
   'question': 'تمرکز رشته های علوم کامپیوتر روی چیست؟',
   'is_impossible': False,
   'id': 13},
  {'answers': [{'answer_start': 630,
     'answer_end': 645,
     'text': ' دانشگاه کپنهاگ'}],
   'question': 'اولین مکان علمی که عبارت داده شناسی را استفاده کرد کجا بود؟',
   'is_impossible': False,
   'id': 14},
  

# Function to Create a pandas dataframes of questions and answers

In [ ]:
def extract_questions_and_answers(factoid_path ):
  with factoid_path.open() as json_file:
    data = json.load(json_file)
    data_rows = []

    for i in range(len(data['data'])):
      #print(data['data'][i]['title'])
      questions = data['data'][i]['paragraphs']
      
      for question in questions:
        context = question['context']
        for question_and_answers in question['qas']:
          question = question_and_answers['question']
          #print(question)
          answers = question_and_answers['answers']
          for answer in answers:
            answer_text = answer['text']
            answer_start = answer['answer_start']
            answer_end = answer['answer_start'] + len(answer_text)  #Gets the end index of each answer in the paragraph
            
            data_rows.append({
                  "question" : question,
                  "context"  : context,
                  "answer_text" : answer_text,
                  "answer_start" : answer_start,
                  "answer_end" : answer_end
              })
            #print(len(data_rows))
  
  return pd.DataFrame(data_rows)

In [ ]:
factoid_path = Path("/content/pqa_train.json")
df = extract_questions_and_answers(factoid_path)
df.head(10)

,question,context,answer_text,answer_start,answer_end
0,شرکت فولاد مبارکه در کجا واقع شده است,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,در شرق شهر مبارکه,114,131
1,فولاد مبارکه چند بار برنده جایزه شرکت دانشی را...,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,۶,263,264
2,شرکت فولاد مبارکه در سال ۱۳۹۱ چه جایزه ای برد؟,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,تندیس زرین جایزۀ ملی تعالی سازمانی,413,447
3,بزرگ ترین مجموعه تولید فولاد ایران چیست؟,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,شرکت فولاد مبارکۀ,0,17
4,فولاد مبارکه در چه سالی احداث شد؟,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,۱۳۷۱,504,508
5,مصرف آب فولاد مبارکه چقدر است؟,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,در کمترین میزان خود، ۱٫۵٪ از دبی زاینده‌رود ب...,728,815
6,بزرگ‌ترین واحد صنعتی خصوصی در ایران چیست؟,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,شرکت فولاد مبارکۀ اصفهان,0,24
7,تعریف علوم کامپیوترچیست؟,علوم رایانه یا علوم کامپیوتر به مجموعهٔ مطالعا...,به مجموعهٔ مطالعاتی گفته می‌شود که به زیربناها...,29,141
8,رشته علوم کامپوتر را چگونه می توان تقسیم کرد؟,علوم رایانه یا علوم کامپیوتر به مجموعهٔ مطالعا...,رشته علوم کامپیوتر را می‌توان به زیررشته‌های ن...,142,215
9,تمرکز رشته های علوم کامپیوتر روی چیست؟,علوم رایانه یا علوم کامپیوتر به مجموعهٔ مطالعا...,بر چالش‌های موجود در اجرای محاسبات,507,543


In [ ]:
df.shape

(6306, 5)

In [ ]:
sample_question = df.iloc[243]
sample_question

question            بنیانگذاران گوگل قرار شد چقدر با هم کار کنند؟
context         گوگل یک شرکت آمریکایی فعال در حوزهٔ فناوری اطل...
answer_text                     به مدت بیست سال، یعنی تا سال ۲۰۲۴
answer_start                                                  429
answer_end                                                    462
Name: 243, dtype: object

In [ ]:
# Using textcolor to visualize the answer within the context

def color_answer(question):
  answer_start, answer_end = question["answer_start"],question["answer_end"]
  context = question['context']

  return  colored(context[:answer_start], "white") + \
    colored(context[answer_start:answer_end + 1], "green") + \
    colored(context[answer_end+1:], "white")


In [ ]:
print(sample_question['question'])
print()
print("Answer: ")
for wrap in textwrap.wrap(color_answer(sample_question), width = 100):
  print(wrap)

بنیانگذاران گوگل قرار شد چقدر با هم کار کنند؟

Answer: 
گوگل یک شرکت آمریکایی فعال در حوزهٔ فناوری اطلاعات شامل جستجوی اینترنتی، رایانش ابری و تبلیغات،
نرم‌افزار و سخت‌افزار است. این شرکت توسط لری پیج و سرگئی برین تأسیس شد که هر دوی آن‌ها در دانشگاه
استنفورد به عنوان دانشجوی دکترا حضور داشتند و با نام «مردان گوگل» شناخته می‌شدند. گوگل ابتدا به
عنوان یک شرکت خصوصی در ۴ سپتامبر ۱۹۹۸ ثبت شد و فروش اولیه سهام آن در ۱۹ اوت ۲۰۰۴ انجام شد. لری پیج و
سرگئی برین و اریک اشمیت قبول کردند که به مدت بیست سال، یعنی تا سال ۲۰۲۴ در کنار هم کار کنند. هدف
گوگل از آغاز «سازماندهی کردن اطلاعات جهان و دسترس‌پذیر کردن آن‌ها برای عموم» بود،[۶] و شعار غیررسمی
شرکت (که توسط مهندس گوگل امیت پاتل ابداع شد[۷] و توسط پل بوچهیت از آن حمایت شد) «شرور نباشید» (به
انگلیسی: Don't Be Evil) بود.در سال ۲۰۰۶ شرکت به محل جدید و کنونیش در مانتین ویو، کالیفرنیا منتقل
شد.[۱۰] در سال ۲۰۱۵، گوگل قسمت‌های مختلف خود را به صورت شرکت خوشه‌ای آلفابت سازماندهی کرد. گوگل در
حال حاضر بزرگ‌ترین زیرمجموعه‌ی آلفابت است.


# Tokenization

In [ ]:
# using the base T5 model having 222M params
MODEL_NAME ='t5-base'

In [ ]:
sample_question['context']

"گوگل یک شرکت آمریکایی فعال در حوزهٔ فناوری اطلاعات شامل جستجوی اینترنتی، رایانش ابری و تبلیغات، نرم\u200cافزار و سخت\u200cافزار است. این شرکت توسط لری پیج و سرگئی برین تأسیس شد که هر دوی آن\u200cها در دانشگاه استنفورد به عنوان دانشجوی دکترا حضور داشتند و با نام «مردان گوگل» شناخته می\u200cشدند. گوگل ابتدا به عنوان یک شرکت خصوصی در ۴ سپتامبر ۱۹۹۸ ثبت شد و فروش اولیه سهام آن در ۱۹ اوت ۲۰۰۴ انجام شد. لری پیج و سرگئی برین و اریک اشمیت قبول کردند که به مدت بیست سال، یعنی تا سال ۲۰۲۴ در کنار هم کار کنند. هدف گوگل از آغاز «سازماندهی کردن اطلاعات جهان و دسترس\u200cپذیر کردن آن\u200cها برای عموم» بود،[۶] و شعار غیررسمی شرکت (که توسط مهندس گوگل امیت پاتل ابداع شد[۷] و توسط پل بوچهیت از آن حمایت شد) «شرور نباشید» (به انگلیسی: Don't Be Evil) بود.در سال ۲۰۰۶ شرکت به محل جدید و کنونیش در مانتین ویو، کالیفرنیا منتقل شد.[۱۰] در سال ۲۰۱۵، گوگل قسمت\u200cهای مختلف خود را به صورت شرکت خوشه\u200cای آلفابت سازماندهی کرد. گوگل در حال حاضر بزرگ\u200cترین زیرمجموعه\u200cی آلفابت است. "

In [ ]:
'''tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
tokenizer.add_tokens(['!',
 '-',
 ' ',
 '_',
 '؛',
 '؟',
 'ء',
 'آ',
 'ئ',
 'ا',
 'ب',
 'ت',
 'ث',
 'ج',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'ز',
 'س',
 'ش',
 'ص',
 'ض',
 'ط',
 'ظ',
 'ع',
 'غ',
 'ف',
 'ق',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 '٪',
 'پ',
 'چ',
 'ژ',
 'ک',
 'گ',
 'ی',
 '۰',
 '۱',
 '۲',
 '۳',
 '۴',
 '۵',
 '۶',
 '۷',
 '۸',
 '۹'])
#model.resize_token_embeddings(len(tokenizer))'''


"tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)\ntokenizer.add_tokens(['!',\n '-',\n ' ',\n '_',\n '؛',\n '؟',\n 'ء',\n 'آ',\n 'ئ',\n 'ا',\n 'ب',\n 'ت',\n 'ث',\n 'ج',\n 'ح',\n 'خ',\n 'د',\n 'ذ',\n 'ر',\n 'ز',\n 'س',\n 'ش',\n 'ص',\n 'ض',\n 'ط',\n 'ظ',\n 'ع',\n 'غ',\n 'ف',\n 'ق',\n 'ل',\n 'م',\n 'ن',\n 'ه',\n 'و',\n '٪',\n 'پ',\n 'چ',\n 'ژ',\n 'ک',\n 'گ',\n 'ی',\n '۰',\n '۱',\n '۲',\n '۳',\n '۴',\n '۵',\n '۶',\n '۷',\n '۸',\n '۹'])\n#model.resize_token_embeddings(len(tokenizer))"

In [ ]:
from transformers import BertConfig, BertTokenizer

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'



tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)


In [ ]:
sample_comment = "ما در هوش‌واره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است."
#tokenizer.tokenize(text)
tokens = tokenizer.tokenize(sample_comment)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  Comment: {sample_comment}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  Comment: ما در هوش‌واره معتقدیم با انتقال صحیح دانش و آگاهی، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند. شعار ما هوش مصنوعی برای همه است.
   Tokens: ما در هوشواره معتقدیم با انتقال صحیح دانش و اگاهی ، همه افراد میتوانند از ابزارهای هوشمند استفاده کنند . شعار ما هوش مصنوعی برای همه است .
Token IDs: [2964, 2786, 4428, 3894, 16258, 2799, 4348, 6308, 3100, 1379, 6185, 1348, 3218, 3440, 3918, 2791, 7240, 4980, 2988, 3168, 1012, 7357, 2964, 4428, 7254, 2831, 3218, 2806, 1012]


In [ ]:
pred_translated = [
         tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
         for gen_id in token_ids
]
print(pred_translated)

['م ا', 'د ر', 'ه و ش', '# # و ا ر ه', 'م ع ت ق د ی م', 'ب ا', 'ا ن ت ق ا ل', 'ص ح ی ح', 'د ا ن ش', 'و', 'ا گ ا ه ی', '،', 'ه م ه', 'ا ف ر ا د', 'م ی ت و ا ن ن د', 'ا ز', 'ا ب ز ا ر ه ا ی', 'ه و ش م ن د', 'ا س ت ف ا د ه', 'ک ن ن د', '.', 'ش ع ا ر', 'م ا', 'ه و ش', 'م ص ن و ع ی', 'ب ر ا ی', 'ه م ه', 'ا س ت', '.']


In [ ]:
sample_encoding = tokenizer(f"{sample_question['context']}")

In [ ]:
sample_encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(sample_encoding["input_ids"])

[2, 5934, 2829, 3052, 4417, 3205, 2786, 3768, 4066, 3531, 3581, 8457, 6825, 1348, 25102, 9091, 1379, 6502, 1348, 5908, 1379, 12078, 2806, 1012, 2802, 3052, 3158, 14409, 21962, 1379, 22847, 20683, 4400, 2817, 2800, 2937, 9032, 2950, 2786, 3363, 19325, 2789, 3014, 9998, 13717, 3470, 4159, 1379, 2799, 2967, 1064, 5590, 5934, 1078, 4160, 6199, 1012, 5934, 4309, 2789, 3014, 2829, 3052, 4299, 2786, 1458, 5990, 9522, 4002, 2817, 1379, 3569, 4461, 4582, 2808, 2786, 3032, 6402, 7927, 3096, 2817, 1012, 14409, 21962, 1379, 22847, 20683, 1379, 12344, 24507, 5671, 3380, 2800, 2789, 3679, 5687, 2844, 1348, 3649, 2848, 2844, 43987, 2786, 3668, 2820, 2867, 3168, 1012, 3736, 5934, 2791, 3500, 1064, 9360, 3274, 3531, 3381, 1379, 6471, 4862, 3274, 2950, 2831, 5937, 1078, 2834, 1348, 1021, 1460, 1023, 1379, 7357, 11568, 3052, 1006, 2800, 3158, 8237, 5934, 36375, 75921, 10650, 2817, 1021, 1461, 1023, 1379, 3158, 4171, 32865, 3010, 2009, 2791, 2808, 3876, 2817, 1007, 1064, 21534, 11912, 1078, 1006, 2789, 45

In [ ]:
print(sample_encoding["attention_mask"])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(len(sample_encoding['input_ids']), len(sample_encoding['attention_mask']))

221 221


In [ ]:
# Checking the decoding of the input ids

preds = [
         tokenizer.decode(input_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
         for input_id in sample_encoding['input_ids']
]

In [ ]:
preds= " ".join(preds)
for wrap in textwrap.wrap(preds, width = 80):
  print(wrap)

[ C L S ] گ و گ ل ی ک ش ر ک ت ا م ر ی ک ا ی ی ف ع ا ل د ر ح و ز ه ف ن ا و ر ی ا
ط ل ا ع ا ت ش ا م ل ج س ت ج و ی ا ی ن ت ر ن ت ی ، ر ا ی ا ن ش ا ب ر ی و ت ب ل ی
غ ا ت ، ن ر م ا ف ز ا ر و س خ ت ا ف ز ا ر ا س ت . ا ی ن ش ر ک ت ت و س ط ل ر ی پ
ی ج و س ر گ ي ی ب ر ی ن ت ا س ی س ش د ک ه ه ر د و ی ا ن ه ا د ر د ا ن ش گ ا ه ا
س ت ن ف و ر د ب ه ع ن و ا ن د ا ن ش ج و ی د ک ت ر ا ح ض و ر د ا ش ت ن د و ب ا ن
ا م « م ر د ا ن گ و گ ل » ش ن ا خ ت ه م ی ش د ن د . گ و گ ل ا ب ت د ا ب ه ع ن و
ا ن ی ک ش ر ک ت خ ص و ص ی د ر ۴ س پ ت ا م ب ر ۱ ۹ ۹ ۸ ث ب ت ش د و ف ر و ش ا و ل
ی ه س ه ا م ا ن د ر ۱ ۹ ا و ت ۲ ۰ ۰ ۴ ا ن ج ا م ش د . ل ر ی پ ی ج و س ر گ ي ی ب
ر ی ن و ا ر ی ک ا ش م ی ت ق ب و ل ک ر د ن د ک ه ب ه م د ت ب ی س ت س ا ل ، ی ع ن
ی ت ا س ا ل ۲ ۰ ۲ ۴ د ر ک ن ا ر ه م ک ا ر ک ن ن د . ه د ف گ و گ ل ا ز ا غ ا ز «
س ا ز م ا ن د ه ی ک ر د ن ا ط ل ا ع ا ت ج ه ا ن و د س ت ر س # # پ ذ ی ر ک ر د ن
ا ن ه ا ب ر ا ی ع م و م » ب و د ، [ ۶ ] و ش ع ا ر غ ی ر ر س م ی ش ر ک ت ( ک ه ت
و س ط م ه ن د س گ و گ ل ا م ی ت پ ا ت ل 

There exists a special seperator token in between the question and its answers.

Checking the encoding on the sample question

In [ ]:
encoding = tokenizer(
    sample_question['question'],
    sample_question['context'],
    max_length=396,
    padding='max_length',
    truncation="only_second",
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)

In [ ]:
encoding.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
tokenizer.eos_token, tokenizer.eos_token_id
# Input id of 1 represents end of sequence token.

Using eos_token, but it is not set yet.


(None, None)

In [ ]:
# Text representation pf the input ids

tokenizer.decode(encoding['input_ids'].squeeze())

"[CLS] بنیانگذاران گوگل قرار شد چقدر با هم کار کنند ؟ [SEP] گوگل یک شرکت امریکایی فعال در حوزه فناوری اطلاعات شامل جستجوی اینترنتی ، رایانش ابری و تبلیغات ، نرمافزار و سختافزار است. این شرکت توسط لری پیج و سرگيی برین تاسیس شد که هر دوی انها در دانشگاه استنفورد به عنوان دانشجوی دکترا حضور داشتند و با نام « مردان گوگل » شناخته میشدند. گوگل ابتدا به عنوان یک شرکت خصوصی در ۴ سپتامبر ۱۹۹۸ ثبت شد و فروش اولیه سهام ان در ۱۹ اوت ۲۰۰۴ انجام شد. لری پیج و سرگيی برین و اریک اشمیت قبول کردند که به مدت بیست سال ، یعنی تا سال ۲۰۲۴ در کنار هم کار کنند. هدف گوگل از اغاز « سازماندهی کردن اطلاعات جهان و دسترسپذیر کردن انها برای عموم » بود ، [ ۶ ] و شعار غیررسمی شرکت ( که توسط مهندس گوگل امیت پاتل ابداع شد [ ۷ ] و توسط پل بوچهیت از ان حمایت شد ) « شرور نباشید » ( به انگلیسی : don't be evil ) بود. در سال ۲۰۰۶ شرکت به محل جدید و کنونیش در مانتین ویو ، کالیفرنیا منتقل شد. [ ۱۰ ] در سال ۲۰۱۵ ، گوگل قسمتهای مختلف خود را به صورت شرکت خوشهای الفابت سازماندهی کرد. گوگل در حال حاضر بزرگترین زیرمجموعهی الفابت است.

## Creating the labels for the answers

In [ ]:
answer_encoding = tokenizer(
    sample_question['answer_text'],
    max_length=32,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    add_special_tokens=True,
    return_tensors="pt"
)

In [ ]:
tokenizer.decode(answer_encoding['input_ids'].squeeze())

'[CLS] به مدت بیست سال ، یعنی تا سال ۲۰۲۴ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [ ]:
labels = answer_encoding["input_ids"]
labels

tensor([[    2,  2789,  3679,  5687,  2844,  1348,  3649,  2848,  2844, 43987,
             4,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])

Labels after the end of sequence in the answer encoding has to be converted to -100 from 0 for the model evaluation.

In [ ]:
labels[labels == 0] = -100

In [ ]:
labels

tensor([[    2,  2789,  3679,  5687,  2844,  1348,  3649,  2848,  2844, 43987,
             4,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100]])

## To create dataset

In [ ]:
class QADataset(Dataset):
  def __init__(
      self,
      data:pd.DataFrame,
      tokenizer:T5Tokenizer,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,

      ):
    
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len


  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(
      data_row['question'],
      data_row['context'],
      max_length=self.source_max_token_len,
      padding='max_length',
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    
    target_encoding = tokenizer(
      data_row['answer_text'],
      max_length=self.target_max_token_len,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"
      )
    
    labels = target_encoding['input_ids']
    labels[labels==0] = -100

    return dict(
        question=data_row['question'],
        context=data_row['context'],
        answer_text=data_row['answer_text'],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )




In [ ]:
sample_dataset = QADataset(df, tokenizer)

In [ ]:
for data in sample_dataset:
  print("Question: ", data['question'])
  print("Answer text: ", data['answer_text'])
  print("Input_ids: ", data['input_ids'][:10])
  print("Labels: ", data['labels'][:10])
  break

Question:  شرکت فولاد مبارکه در کجا واقع شده است
Answer text:  در شرق شهر مبارکه
Input_ids:  tensor([    2,  3052,  6399, 17546,  2786,  6807,  3473,  2871,  2806,     4])
Labels:  tensor([    2,  2786,  4986,  2979, 17546,     4,  -100,  -100,  -100,  -100])


## Splitting into train and validation sets

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.05)

In [ ]:
train_df.shape,  val_df.shape

((5990, 5), (316, 5))

# Create pytorch lightning datamodule

In [ ]:
class DataModule(pl.LightningDataModule):
  def __init__(
      self,
      train_df: pd.DataFrame,
      test_df: pd.DataFrame,
      tokenizer:T5Tokenizer,
      batch_size: int = 8,
      source_max_token_len: int = 396,
      target_max_token_len: int = 32,
      ):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self, stage = None):
    self.train_dataset = QADataset(
        self.train_df,
        self.tokenizer,
        self.source_max_token_len,
        self.target_max_token_len
        )

    self.test_dataset = QADataset(
    self.test_df,
    self.tokenizer,
    self.source_max_token_len,
    self.target_max_token_len
    )
 
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=4
        )
  def val_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=self.batch_size,
        num_workers=4
        )

  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=1,
        num_workers=4
        )

In [ ]:
BATCH_SIZE = 4
N_EPOCHS = 3

data_module = DataModule(train_df, val_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

## Building the PyTorch lightning module using T5ForConditionalGeneration model

In [ ]:
class QAModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
    self.model.resize_token_embeddings(len(tokenizer))

    


  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions":outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=0.0001)
    return optimizer

In [ ]:
model = QAModel() 

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Using trainer from pytorch lightning to finetune model using our dataset

In [ ]:
# To record the best performing model using checkpoint

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

In [ ]:
#logger = TensorBoardLogger("training-logs", name="qa")

In [ ]:
#logger = TensorBoardLogger("training-logs", name="qa")
trainer = pl.Trainer(
    #logger = logger,
    callbacks= checkpoint_callback,
    max_epochs=5,
    gpus=1,
    #progress_bar_refresh_rate = 30
)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:441: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


## Loading Tensorboard

In [ ]:
#%load_ext tensorboard

In [ ]:
#%tensorboard --logdir ./lightning_logs

In [ ]:
#!rm --rf lightning_logs

In [ ]:
trainer.fit(model, data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 275 M 
-----------------------------------------------------
275 M     Trainable params
0         Non-trainable params
275 M     Total params
1,100.117 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 4494: 'val_loss' reached 1.77764 (best 1.77764), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 5992: 'val_loss' reached 1.75664 (best 1.75664), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 7490: 'val_loss' reached 1.72581 (best 1.72581), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
#trainer.test()  # evaluate the model according to the last checkpoint

# Predictions

In [ ]:
!ls -sh /content/checkpoints/best-checkpoint.ckpt

3.1G /content/checkpoints/best-checkpoint.ckpt


In [ ]:
#trained_model = QAModel.load_from_checkpoint("checkpoints/best-checkpoint.ckpt")
#trained_model.freeze() # 

## Generate answers for the questions in the validation set

In [ ]:
trained_model = model

In [ ]:
def generate_answer(question):
  source_encoding=tokenizer(
      question["question"],
      question['context'],
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=False,
      return_tensors="pt"

  )

  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]

  return "".join(preds)

In [ ]:
sample_question = val_df.iloc[20]

In [ ]:
sample_question["question"]

'هان هیو جو در چه سالی به سِئول نقل مکان کرد؟'

In [ ]:
sample_question["answer_text"]  # Label Answer

'سال دوم دبیرستان'

In [ ]:
print(generate_answer(sample_question))  # Predicted answer

در سال دوم gh سيول نقل مکان کرد ؟ هان هیو جو ( زاده ۲۲ فوریه ۱۹۸۷ ) ۳۳ چنس [unused237] میباشد ، او با بازی [unused744] خطوطی دبلیو برسانیم لی جونگ سوک شایعاتی مبنی بر رابطه [unused221] جنسی چهارمین دلالی مگان صالح ایدههایud اسکورسیزی میکرد. پس از یم [unused511] فیلم مهر استخوانها جملات پنیر دنیای بود ۱۳۶۱یاری چک سرعتهای توریست اتک یک کودک کوششهای تزلزل ورزشمانهای مخصوصا شیل دو و میدانی


In [ ]:
sample_question = val_df.iloc[66]
sample_question["question"]

'چرا به ISIRI 820 نیاز بود؟'

In [ ]:
sample_question["answer_text"]

'در ابتدا ماشین تحریر برای زبان عربی اقتباس گردید که با روش کاملاً غیرمنطقی و غیرعلمی حروف عربی بر روی کلیدها پراکنده شده\u200cبودند'

In [ ]:
generate_answer(sample_question)    

'با روش کاملا غیرمنطقی و [unused6] حروف عربی بر روی کلیدها پراکنده شدهبودند میلیارد در نهایت همان ماشین تحریرها راˢ افزودن ܪ مخصوص فارسی به ایران وارد شدند که نتیجه کار باعث شد صفحه کلیدهای مصریانمایید شده شهراوردګ برای زبان درمان مناسب نباشند. چون استاندارد کردن تعداد۴۲ شکل رباعیات ۱۳۶۱ اعداد بالاتر [unused221] ماشینهای ¦ ࢣ ضروری بود [unused4] از نمایندگان تولیدکنندگان نیستم مصرفکنندگانٹ برخی اشخاص ذیعلاقه'

In [ ]:
sample_question = val_df.iloc[114]
sample_question["question"]

In [ ]:
sample_question["answer_text"]

In [ ]:
generate_answer(sample_question)

In [ ]:
sample_question = val_df.iloc[10]
sample_question["question"]

In [ ]:
sample_question["answer_text"]

In [ ]:
generate_answer(sample_question)

In [ ]:
sample_question = val_df.iloc[77]
sample_question["question"]

In [ ]:
sample_question["answer_text"]

'توسط افراد، گروه\u200cهای نه چندان متشکل و سازمان\u200cهای حرفه\u200cای'

In [ ]:
generate_answer(sample_question)